In [6]:
from __future__ import division
from sympy import *

# For printing sympy objects
from IPython.display import display
# This changes the output of display to use mathjax which displays 
# nice math objects (looks like latex).
init_printing(use_latex='mathjax')

import plotly
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
tls.set_credentials_file(username='calroy', api_key='b7nwwqa95s')


print plotly.__version__            # version 1.9.x required
plotly.offline.init_notebook_mode() # run at the start of every notebook
import math
import pandas as pd
import numpy as np

from scipy import signal
from scipy.signal import butter, lfilter, freqz
import scipy.fftpack

1.9.6


# Find the hydraulic power required by the roll proportional valve
NC_2P8m_160519105336_slalems tight turns test.bin.tab

## Calculate Power Required from Roll in system
How much instantaneous power was 2play using during this test?  

Hydraulic Power

kW	=	Flow rate (L/min) x diff pressure (bar) / 600

For the proportional valve their are two differential pressures across the valve but they are about equal at all times.

The differential pressures depend on which way the valve is open. The four possible differential pressures are;  
  high_r_across_pitch_valve_1 = rollProValvePressure - rdblp  
  high_r_across_pitch_valve_2 = rollProValvePressure - rdbrp  
  low_r_across_pitch_valve_2 = rdblp  
  low_r_across_pitch_valve_1 = rdbrp  


Just take the smallest of either rdblp or rdbrp and use this as the differential pressure across the valve for one side of the valve to calcualte hydraulic power. The other differential pressure is the other side of the valve which will have a differential pressure of 100 bar - supply pressure reading.

This is the differential pressures 

In [17]:
nautiData = pd.read_csv(r'./NC_2P8m_160519105336_slalems tight turns test.bin.tab', delimiter='\t')
print list(nautiData.columns.values)

logtime = nautiData['logtime']
# Calculate the roll dumbbell flow
rdbx_vel = nautiData['rdbx']
rdbx_vel =np.gradient(rdbx_vel, 0.04)
# 0.00282743338823 Liters moved in the inside of the db per mm travel of roll dumbbell.
# This is the flow moved in the outer dumbbell circuit pushed by the top area of the dumbbell.
roll_proportional_valve_flow = rdbx_vel* 0.00282743338823
# Convert to L/min
roll_proportional_valve_flow = roll_proportional_valve_flow*60


INS_Roll = nautiData['INS_Roll']
INS_Roll_deg = 57.3 * INS_Roll
INS_Yaw = nautiData['INS_Yaw']
INS_Yaw_deg = INS_Yaw * 57.3
INS_Yaw_rate = np.diff(INS_Yaw_deg, 0.04)

steering_angle = nautiData['steering_angle']

# 0.00785398 Liters moved in outside of the db per mm travel of roll dumbbell.
# 0.00282743338823 Liters moved in the inside of the db per mm travel of roll dumbbell.
# This is the flow moved in the outer dumbbell circuit pushed by the top area of the dumbbell.
roll_proportional_valve_flow = rdbx_vel* 0.00282743338823
# Convert to L/min
roll_proportional_valve_flow = roll_proportional_valve_flow*60

trace1 = Scatter(x=logtime, y=nautiData['rdbx'], name='rdbx', mode='lines', yaxis='y')
trace2 = Scatter(x=logtime, y=nautiData['craft_speed'], name='_NMEA_SPEED', mode='lines', yaxis='y2')
trace3 = Scatter(x=logtime, y=roll_proportional_valve_flow, name='roll_proportional_valve_flow', mode='lines', yaxis='y3')
trace4 = Scatter(x=logtime, y=INS_Roll_deg, name='roll angle', mode='lines', yaxis='y4')
trace5 = Scatter(x=logtime, y=INS_Yaw_rate, name='INS_Yaw_rate', mode='lines', yaxis='y5')
trace6 = Scatter(x=logtime, y=steering_angle, name='steering_angle', mode='lines', yaxis='y4')



data = Data([trace1, trace2, trace3, trace4, trace5, trace6])

# Construct the Layout object
xaxis = XAxis(title='Seconds')
yaxis = YAxis(title='rdbx mm')
yaxis2 = YAxis(title='_NMEA_SPEED knots',
              overlaying='y',
               side='right',
               anchor = 'free',
               position = 1.0 )

yaxis3 = YAxis(title='roll_proportional_valve_flow L/min',
              overlaying='y',
               side='left',
               anchor = 'free',
               position = 0.08 )
yaxis4 = YAxis(title='roll_angle degs',
              overlaying='y',
               side='left',
               anchor = 'free',
               position = 0.13 )

yaxis5 = YAxis(title='yaw degs',
              overlaying='y',
               side='right',
               anchor = 'free',
               position = 0.92 )


layout = Layout(
    title='fluid flows',
    xaxis1=xaxis,
    yaxis = yaxis,
    yaxis2 = yaxis2,
    yaxis3 = yaxis3,
    yaxis4 = yaxis4,
    yaxis5 = yaxis5
)

fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

['\xef\xbb\xbf_ID', 'logtime', 'flx', 'frx', 'rlx', 'rrx', 'pdbx', 'rdbx', 'fldx', 'frdx', 'rldx', 'rrdx', 'pfp', 'prp', 'pfdbp', 'prdbp', 'rlp', 'rrp', 'rldbp', 'rrdbp', 'steering_angle', 'craft_speed', 'craft_acceleration', 'pitchfrontin', 'pitchfrontout', 'pitchrearin', 'pitchrearout', 'pitchDBfrontin', 'pitchDBfrontout', 'pitchDBrearin', 'pitchDBrearout', 'pitchDBfrontacclock', 'pitchDBrearacclock', 'pitchDBfrontactivelock', 'pitchDBrearactivelock', 'rollleftin', 'rollleftout', 'rollrightin', 'rollrightout', 'rollDBleftin', 'rollDBleftout', 'rollDBrightin', 'rollDBrightout', 'rollDBleftacclock', 'rollDBrightacclock', 'rollDBleftactivelock', 'rollDBrightactivelock', 'damperFLcompression', 'damperFLrebound', 'damperFRcompression', 'damperFRrebound', 'damperRLcompression', 'damperRLrebound', 'damperRRcompression', 'damperRRrebound', 'pitchactive', 'rollactive', 'req_pitch_flow_rate', 'req_roll_flow_rate', 'manual_active', 'setup_active', 'damping_active', 'active_active', 'dump_active

In [29]:
nautiData = pd.read_csv(r'./NC_2P8m_160519105336_slalems tight turns test.bin.tab', delimiter='\t')
print list(nautiData.columns.values)

logtime = nautiData['logtime']
# Calculate the roll dumbbell flow
rdbx_vel = nautiData['rdbx']
rdbx_vel =np.gradient(rdbx_vel, 0.04)
# 0.00282743338823 Liters moved in the inside of the db per mm travel of roll dumbbell.
# This is the flow moved in the outer dumbbell circuit pushed by the top area of the dumbbell.
roll_proportional_valve_flow = rdbx_vel* 0.00282743338823
# Convert to L/min
roll_proportional_valve_flow = roll_proportional_valve_flow*60

rdb_total_flow = scipy.integrate.cumtrapz(roll_proportional_valve_flow,dx=0.04)
rdbx = nautiData['rdbx']
rdblp = nautiData['rldbp']
rdbrp = nautiData['rrdbp']
low_diff_rvalve = np.zeros_like(rdbrp)
high_diff_rvalve = np.zeros_like(rdbrp)
diff_rdbp = np.zeros_like(rdbrp)
supply = nautiData['rollProValvePressure'] # 100 bar since it was not logged for this data set.

for i in range(len(rdbrp)):
    if rdbrp[i] < rdblp[i]:
        low_diff_rvalve[i] = rdbrp[i]
        high_diff_rvalve[i] = supply[i] - rdblp[i]
        diff_rdbp[i] = rdblp[i] - rdbrp[i]
    else:
        low_diff_rvalve[i] = rdblp[i]
        high_diff_rvalve[i] = supply[i] - rdbrp[i]
        diff_rdbp[i] = rdbrp[i] - rdblp[i]
        

  
# This is the flow moved in the outer dumbbell circuit pushed by the top area of the dumbbell.
# Calculate the roll dumbbell flow
rdbx_vel = nautiData['rdbx']
rdbx_vel =np.gradient(rdbx_vel, 0.04)
roll_proportional_valve_flow = rdbx_vel* 0.00282743338823
# Convert to L/min
roll_proportional_valve_flow = roll_proportional_valve_flow*60

# Hydraulic power kW put into the roll proportional valve
hyd_power_rvalve = abs((low_diff_rvalve + high_diff_rvalve) * roll_proportional_valve_flow) / 600

# Hydraulic power put into the roll dumbbell
hyd_power_rdb = abs(diff_rdbp * roll_proportional_valve_flow) / 600 

# Total Hydraulic power put into Entire roll in system
hyd_power_r_total = abs(supply * roll_proportional_valve_flow) / 600
    
trace1 = Scatter(x=logtime, y=rdbx, name='rdbx', mode='lines', yaxis='y')
trace2 = Scatter(x=logtime, y=roll_proportional_valve_flow, name='roll_proportional_valve_flow', mode='lines', yaxis='y3')
trace3 = Scatter(x=logtime, y=rdblp, name='rdblp', mode='lines', yaxis='y2')
trace4 = Scatter(x=logtime, y=rdbrp, name='rdbrp', mode='lines', yaxis='y2')
trace5 = Scatter(x=logtime, y=hyd_power_rvalve, name='hyd_power_rvalve', mode='lines', yaxis='y4')
trace6 = Scatter(x=logtime, y=hyd_power_rdb, name='hyd_power_rdb', mode='lines', yaxis='y4')
trace7 = Scatter(x=logtime, y=hyd_power_r_total, name='hyd_power_r_total', mode='lines', yaxis='y4')


data = Data([trace1, trace2, trace3, trace4, trace5, trace6, trace7]) 

# Construct the Layout object
xaxis = XAxis(title='Seconds')
yaxis = YAxis(title='rpdbx mm')
yaxis2 = YAxis(title='pressure bar',
              overlaying='y',
               side='right',
               anchor = 'free',
               position = 1.0 )

yaxis3 = YAxis(title='roll_proportional_valve_flow Liters',
              overlaying='y',
               side='left',
               anchor = 'free',
               position = 0.08 )
yaxis4 = YAxis(title='power Kw',
              overlaying='y',
               side='left',
               anchor = 'free',
               position = 0.13 )


layout = Layout(
    title='Roll Flow/Power',
    xaxis1=xaxis,
    yaxis = yaxis,
    yaxis2 = yaxis2,
    yaxis3 = yaxis3,
    yaxis4 = yaxis4
)

fig = Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

['\xef\xbb\xbf_ID', 'logtime', 'flx', 'frx', 'rlx', 'rrx', 'pdbx', 'rdbx', 'fldx', 'frdx', 'rldx', 'rrdx', 'pfp', 'prp', 'pfdbp', 'prdbp', 'rlp', 'rrp', 'rldbp', 'rrdbp', 'steering_angle', 'craft_speed', 'craft_acceleration', 'pitchfrontin', 'pitchfrontout', 'pitchrearin', 'pitchrearout', 'pitchDBfrontin', 'pitchDBfrontout', 'pitchDBrearin', 'pitchDBrearout', 'pitchDBfrontacclock', 'pitchDBrearacclock', 'pitchDBfrontactivelock', 'pitchDBrearactivelock', 'rollleftin', 'rollleftout', 'rollrightin', 'rollrightout', 'rollDBleftin', 'rollDBleftout', 'rollDBrightin', 'rollDBrightout', 'rollDBleftacclock', 'rollDBrightacclock', 'rollDBleftactivelock', 'rollDBrightactivelock', 'damperFLcompression', 'damperFLrebound', 'damperFRcompression', 'damperFRrebound', 'damperRLcompression', 'damperRLrebound', 'damperRRcompression', 'damperRRrebound', 'pitchactive', 'rollactive', 'req_pitch_flow_rate', 'req_roll_flow_rate', 'manual_active', 'setup_active', 'damping_active', 'active_active', 'dump_active